Team Memebers:
Brinda Rao,
Janusz Feigel,
Bhavana Malla

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#prepare the train data
max_words = 20000
max_len = 200

(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

def gen():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

train_data = tf.data.Dataset.from_generator(gen, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
buckets = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
bucket_batch_size = [32] * (len(buckets) + 1)
train_data = train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                  bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)
#prepare the test data
def gen_test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

test_data = tf.data.Dataset.from_generator(gen_test, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
test_data = test_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0], bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)

#define the model
# embedding, masking and keras LSTM
model = tf.keras.Sequential([tf.keras.layers.Embedding(max_words, 128, mask_zero=True), 
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#run the model 
history = model.fit(train_data, epochs=5)

test_scores = model.evaluate(test_data)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/5
786/786 [==============================] - 126s 143ms/step - loss: 0.4246 - accuracy: 0.8059
Epoch 2/5
786/786 [==============================] - 111s 141ms/step - loss: 0.3329 - accuracy: 0.8599
Epoch 3/5
786/786 [==============================] - 111s 141ms/step - loss: 0.2672 - accuracy: 0.8980
Epoch 4/5
786/786 [==============================] - 114s 145ms/step - loss: 0.1605 - accuracy: 0.9441
Epoch 5/5
787/787 [==============================] - 53s 62ms/step - loss: 0.4077 - accuracy: 0.8541
Test loss: 0.40771278738975525
Test accuracy: 0.8541200160980225


#Embedding and LSTM Experiments
* Embedding 256, LSTM 128, LSTM 64, 5 epochs: 0.95 train acc, 0.82 test acc
* Embedding 128, LSTM 32, LSTM 32, 5 epochs: 0.93 train acc, 0.86 test acc
* Embedding 128, LSTM 32, LSTM 32, LSTM 32, 5 epochs: 0.97 train acc, 0.86 test acc
* Embedding 256, LSTM 32, LSTM 32, LSTM 32, 5 epochs: 0.90 train acc, 0.86 test acc
* Embedding 128, LSTM 128, LSTM 64, LSTM 32, 5 epochs: 0.92 train acc, 0.86 test acc
* Embedding 128, LSTM 128, LSTM 128, LSTM 128, 5 epochs: 0.93 train acc, 0.86 test acc

#GRU

In [ ]:
#prepare the train data
max_words = 20000
max_len = 200

(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

def gen():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

train_data = tf.data.Dataset.from_generator(gen, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
buckets = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
bucket_batch_size = [32] * (len(buckets) + 1)
train_data = train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                  bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)
#prepare the test data
def gen_test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

test_data = tf.data.Dataset.from_generator(gen_test, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
test_data = test_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0], bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)

#define the model
# embedding, masking and keras LSTM
model = tf.keras.Sequential([tf.keras.layers.Embedding(max_words, 128, mask_zero=True), 
                             tf.keras.layers.GRU(32, return_sequences=True),
                             tf.keras.layers.GRU(32, return_sequences=True),
                             tf.keras.layers.GRU(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#run the model 
history = model.fit(train_data, epochs=5)

test_scores = model.evaluate(test_data)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/5
786/786 [==============================] - 131s 150ms/step - loss: 0.4575 - accuracy: 0.7834
Epoch 2/5
786/786 [==============================] - 118s 150ms/step - loss: 0.2390 - accuracy: 0.9081
Epoch 3/5
786/786 [==============================] - 118s 150ms/step - loss: 0.1654 - accuracy: 0.9408
Epoch 4/5
786/786 [==============================] - 118s 150ms/step - loss: 0.0917 - accuracy: 0.9707
Epoch 5/5
787/787 [==============================] - 55s 65ms/step - loss: 0.4842 - accuracy: 0.8569
Test loss: 0.48423057794570923
Test accuracy: 0.8568800091743469


same speed, better train accuracy, same test accuracy

#Bidirectional

In [ ]:
#prepare the train data
max_words = 20000
max_len = 200

(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

def gen():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

train_data = tf.data.Dataset.from_generator(gen, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
buckets = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
bucket_batch_size = [32] * (len(buckets) + 1)
train_data = train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                  bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)
#prepare the test data
def gen_test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

test_data = tf.data.Dataset.from_generator(gen_test, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
test_data = test_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0], bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)

#define the model
# embedding, masking and keras LSTM
model = tf.keras.Sequential([tf.keras.layers.Embedding(max_words, 128, mask_zero=True), 
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(1, activation="sigmoid")])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#run the model 
history = model.fit(train_data, epochs=5)

test_scores = model.evaluate(test_data)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/5
786/786 [==============================] - 266s 303ms/step - loss: 0.3648 - accuracy: 0.8322
Epoch 2/5
786/786 [==============================] - 237s 302ms/step - loss: 0.1737 - accuracy: 0.9360
Epoch 3/5
786/786 [==============================] - 236s 301ms/step - loss: 0.0918 - accuracy: 0.9682
Epoch 4/5
786/786 [==============================] - 237s 301ms/step - loss: 0.0780 - accuracy: 0.9716
Epoch 5/5
787/787 [==============================] - 105s 123ms/step - loss: 0.5764 - accuracy: 0.8508
Test loss: 0.5764022469520569
Test accuracy: 0.8508399724960327


2 times slower, better train accuracy, similar test accuracy

#Without bucketing

In [ ]:
#prepare the train data
max_words = 20000
max_len = 200
(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

def gen():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

train_data = tf.data.Dataset.from_generator(gen, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

# within-batch padding
train_data = train_data.padded_batch(32)

#prepare the test data
def gen_test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

test_data = tf.data.Dataset.from_generator(gen_test, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

# within-batch padding
test_data = test_data.padded_batch(32)

#define the model
# embedding, masking and keras LSTM
model = tf.keras.Sequential([tf.keras.layers.Embedding(max_words, 128, mask_zero=True), 
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#run the model 
history = model.fit(train_data, epochs=5)

test_scores = model.evaluate(test_data)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/5
782/782 [==============================] - 261s 316ms/step - loss: 0.3897 - accuracy: 0.8240
Epoch 2/5
782/782 [==============================] - 250s 319ms/step - loss: 0.1977 - accuracy: 0.9279
Epoch 3/5
782/782 [==============================] - 249s 319ms/step - loss: 0.1305 - accuracy: 0.9548
Epoch 4/5
782/782 [==============================] - 251s 321ms/step - loss: 0.1106 - accuracy: 0.9613
Epoch 5/5
782/782 [==============================] - 111s 135ms/step - loss: 0.4819 - accuracy: 0.8528
Test loss: 0.4819253981113434
Test accuracy: 0.8527600169181824


similar results, but 2 times slower

#Without bucketing and within-batch padding

In [ ]:
#prepare the train data
max_words = 20000
max_len = 200

(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)


#bucketing and within-batch padding
train_sequences_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len)
train_data = tf.data.Dataset.from_tensor_slices((train_sequences_padded, train_labels))
train_data = train_data.shuffle(25000).batch(64)

#prepare the test data
#without bucketing and within-batch padding

test_sequences_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len)
test_data = tf.data.Dataset.from_tensor_slices((test_sequences_padded, test_labels))
test_data = test_data.shuffle(25000).batch(64)
#define the model
# embedding, masking and keras LSTM
model = tf.keras.Sequential([tf.keras.layers.Embedding(max_words, 128, mask_zero=True), 
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#run the model 
history = model.fit(train_data, epochs=5)

test_scores = model.evaluate(test_data)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/5
391/391 [==============================] - 795s 2s/step - loss: 0.3914 - accuracy: 0.8211
Epoch 2/5
391/391 [==============================] - 775s 2s/step - loss: 0.2018 - accuracy: 0.9245
Epoch 3/5
391/391 [==============================] - 783s 2s/step - loss: 0.1114 - accuracy: 0.9602
Epoch 4/5
391/391 [==============================] - 786s 2s/step - loss: 0.0726 - accuracy: 0.9754
Epoch 5/5
391/391 [==============================] - 60s 140ms/step - loss: 0.5163 - accuracy: 0.8469
Test loss: 0.5163354277610779
Test accuracy: 0.8468800187110901


similar results, 8 times slower

#Without Masking

In [ ]:
#prepare the train data
max_words = 20000
max_len = 200

(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=max_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

def gen():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

train_data = tf.data.Dataset.from_generator(gen, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
buckets = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
bucket_batch_size = [32] * (len(buckets) + 1)
train_data = train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                  bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)
#prepare the test data
def gen_test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

test_data = tf.data.Dataset.from_generator(gen_test, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

#bucketing and within-batch padding
test_data = test_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0], bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)

#define the model
# embedding, masking and keras LSTM
model = tf.keras.Sequential([tf.keras.layers.Embedding(max_words, 128, mask_zero=False), 
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32, return_sequences=True),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#run the model 
history = model.fit(train_data, epochs=5)

test_scores = model.evaluate(test_data)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/5
786/786 [==============================] - 126s 154ms/step - loss: 0.6389 - accuracy: 0.6022
Epoch 2/5
786/786 [==============================] - 118s 150ms/step - loss: 0.6796 - accuracy: 0.5158
Epoch 3/5
786/786 [==============================] - 116s 148ms/step - loss: 0.6945 - accuracy: 0.5031
Epoch 4/5
786/786 [==============================] - 114s 145ms/step - loss: 0.6936 - accuracy: 0.4972
Epoch 5/5
787/787 [==============================] - 58s 71ms/step - loss: 0.6931 - accuracy: 0.5000
Test loss: 0.6931464672088623
Test accuracy: 0.5


similar speed, far worse results